# Open AI Assistant QuickStart
Based on YouTube tutorial on OpenAI Assistants [here](https://www.youtube.com/watch?v=5rcjGjgJNQc)

In [2]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
assistant = client.beta.assistants.create(
    name="Math tutor", 
    instructions="You are a math tutor.  Write and run code to answer math problems.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview" 
)
assistant

In [ ]:
thread = client.beta.threads.create()
thread

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role = "user",
    content = "Solve this problem. 3x + 11 = 14"
    #content = "Solve this problem. What are the values of x in the following equation. 6x + 11 = 23"
    #content = "What are the values of x in the following equation\n\n\ 6x^2 - 11x - 10 = 0"
)
message

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id, 
    instructions="Please start every response with 'Well then'"
)

In [ ]:
# Only continue once the status has changed from "queued" to "complete" or something similar
run.status

In [ ]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
run


In [ ]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
messages

In [ ]:
for message in reversed(messages.data):
    print(message.role, message.content[0].text.value)

Link to files with knowledge retrieval


In [ ]:
file = client.files.create(
    purpose="assistants",
    file=open("resources/strictly-student-guide.pdf", "rb")
)
file

In [ ]:
doc_assistant = client.beta.assistants.create(
    name="Strictly tutor", 
    instructions="Please read the document and answer the questions.",
    tools=[{"type": "file_search"}],
    model="gpt-4-1106-preview" 
    #file_ids=[file.id]
)
doc_assistant

In [ ]:
doc_thread = client.beta.threads.create()
doc_thread

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=doc_thread.id,
    role = "user",
    
    #file_id = file.id,
    #content = "Name the Strictly judges."
    content = "Describe the Strictly TV series."
    #content = "Solve this problem. What are the values of x in the following equation. 6x + 11 = 23"
    #content = "What are the values of x in the following equation\n\n\ 6x^2 - 11x - 10 = 0"
)
message

In [20]:
doc_run = client.beta.threads.runs.create(
    thread_id=doc_thread.id,
    assistant_id=doc_assistant.id
)

In [ ]:
doc_run.status

In [23]:
doc_run = client.beta.threads.runs.retrieve(
    thread_id=doc_thread.id,
    run_id=doc_run.id
)

In [ ]:
doc_run.status

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id=doc_thread.id
)
messages

In [ ]:
for message in reversed(messages.data):
    print(message.role, message.content[0].text.value)